In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

In [21]:
import pandas as pd
data = {
    'text_col': [
        "To jest przykładowe zdanie po polsku.",
        "This is an example sentence in English.",
        "Łatwość korzystania z tego narzędzia jest niesamowita.",
        "The ease of use of this tool is amazing.",
        "Cześć! Jak się masz?",
        "Hello! How are you?"
    ]
}
df = pd.DataFrame(data)
df['target'] = range(1, 7)
df

,text_col,target
0,To jest przykładowe zdanie po polsku.,1
1,This is an example sentence in English.,2
2,Łatwość korzystania z tego narzędzia jest nies...,3
3,The ease of use of this tool is amazing.,4
4,Cześć! Jak się masz?,5
5,Hello! How are you?,6


In [22]:
X_train = df['text_col']
y_train = df['target']

In [56]:
X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]

In [58]:
df['tokens']= (X_train_tokens)

In [73]:
df['tokens']

0    ['To', 'jest', 'przykładowe', 'zdanie', 'po', ...
1    ['This', 'is', 'an', 'example', 'sentence', 'i...
2    ['Łatwość', 'korzystania', 'z', 'tego', 'narzę...
3    ['The', 'ease', 'of', 'use', 'of', 'this', 'to...
4            ['Cześć', '!', 'Jak', 'się', 'masz', '?']
5             ['Hello', '!', 'How', 'are', 'you', '?']
Name: tokens, dtype: object

In [59]:
df

,text_col,target,final,token,tokens
0,To jest przykładowe zdanie po polsku.,1,"['Hello', '!', 'How', 'are', 'you', '?']","[To, jest, przykładowe, zdanie, po, polsku, .]","['To', 'jest', 'przykładowe', 'zdanie', 'po', ..."
1,This is an example sentence in English.,2,"['Hello', '!', 'How', 'are', 'you', '?']","[This, is, an, example, sentence, in, English, .]","['This', 'is', 'an', 'example', 'sentence', 'i..."
2,Łatwość korzystania z tego narzędzia jest nies...,3,"['Hello', '!', 'How', 'are', 'you', '?']","[Łatwość, korzystania, z, tego, narzędzia, jes...","['Łatwość', 'korzystania', 'z', 'tego', 'narzę..."
3,The ease of use of this tool is amazing.,4,"['Hello', '!', 'How', 'are', 'you', '?']","[The, ease, of, use, of, this, tool, is, amazi...","['The', 'ease', 'of', 'use', 'of', 'this', 'to..."
4,Cześć! Jak się masz?,5,"['Hello', '!', 'How', 'are', 'you', '?']","[Cześć, !, Jak, się, masz, ?]","['Cześć', '!', 'Jak', 'się', 'masz', '?']"
5,Hello! How are you?,6,"['Hello', '!', 'How', 'are', 'you', '?']","[Hello, !, How, are, you, ?]","['Hello', '!', 'How', 'are', 'you', '?']"


In [60]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train_tokens)
Train_X_Tfidf = Tfidf_vect.transform(X_train)


In [45]:
Train_X_Tfidf

<6x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [62]:
print(Tfidf_vect.vocabulary_)

{'to': 26, 'jest': 12, 'przykładowe': 20, 'zdanie': 30, 'po': 18, 'polsku': 19, 'this': 25, 'is': 10, 'an': 1, 'example': 6, 'sentence': 21, 'in': 9, 'english': 5, 'łatwość': 31, 'korzystania': 13, 'tego': 23, 'narzędzia': 15, 'niesamowita': 16, 'the': 24, 'ease': 4, 'of': 17, 'use': 28, 'tool': 27, 'amazing': 0, 'cześć': 3, 'jak': 11, 'się': 22, 'masz': 14, 'hello': 7, 'how': 8, 'are': 2, 'you': 29}


In [63]:
print(Train_X_Tfidf)

  (0, 30)	0.41987080495717866
  (0, 26)	0.41987080495717866
  (0, 20)	0.41987080495717866
  (0, 19)	0.41987080495717866
  (0, 18)	0.41987080495717866
  (0, 12)	0.3443000664000145
  (1, 25)	0.32554486881123057
  (1, 21)	0.39699901178248725
  (1, 10)	0.32554486881123057
  (1, 9)	0.39699901178248725
  (1, 6)	0.39699901178248725
  (1, 5)	0.39699901178248725
  (1, 1)	0.39699901178248725
  (2, 31)	0.41987080495717866
  (2, 23)	0.41987080495717866
  (2, 16)	0.41987080495717866
  (2, 15)	0.41987080495717866
  (2, 13)	0.41987080495717866
  (2, 12)	0.3443000664000145
  (3, 28)	0.3109123445439382
  (3, 27)	0.3109123445439382
  (3, 25)	0.25495257018877393
  (3, 24)	0.3109123445439382
  (3, 17)	0.6218246890878764
  (3, 10)	0.25495257018877393
  (3, 4)	0.3109123445439382
  (3, 0)	0.3109123445439382
  (4, 22)	0.5
  (4, 14)	0.5
  (4, 11)	0.5
  (4, 3)	0.5
  (5, 29)	0.5
  (5, 8)	0.5
  (5, 7)	0.5
  (5, 2)	0.5


In [64]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,y_train)

MultinomialNB()

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

In [77]:
def training_pipeline(df_train, df_test):
    X_train = df_train['text_col']
    y_train = df_test['target']
    X_test = df_train['text_col']
    y_test = df_test['target']
    X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(Test_X)
    
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf, y_train)
    predictions_NB = Naive.predict(Test_X_Tfidf)
    print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
    return predictions_NB

In [75]:
training_pipeline(df)

MultinomialNB()